In [0]:
# Drop the schema if exists
spark.sql("drop database if exists dev.swatch_raw cascade");

In [0]:
# Setup the Bronze/Raw Layer

class SetupBronzeLayer:
     def __init__(self):
        self.catalog_name = 'dev'
        self.bronze_schema_name = 'swatch_raw'
        self.device_registration_table_name = 'device_registration'
        self.gym_attendance_table_name = 'gym_attendance'
        self.bpm_table_name = 'bpm'
        self.user_profile_table_name = 'user_profile'
        self.workout_session_table_name = 'workout_session'
        self.device_registration_landing_zone_name = 'device_registration_landing_zone'
        self.gym_attendance_landing_zone_name = 'gym_attendance_landing_zone'

# Create the tables
class CreateBronzeLayerTables(SetupBronzeLayer):
    def __init__(self):
        super().__init__()

    def create_bronze_layer_schema(self):
        spark.sql(f"create database if not exists {self.catalog_name}.{self.bronze_schema_name}")
        print(f"Schema Created - {self.catalog_name}.{self.bronze_schema_name}\n")
    
    def create_device_registration_table(self):
        spark.sql(f"""
                  create table if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_table_name}
                  (
                    user_id string,
                    device_id long,
                    mac_address string,
                    registration_timestamp long
                  ) using delta
                  """)
                
        print(f"Table Created - {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_table_name}")
    
    
    def create_gym_attendance_table(self):
        spark.sql(f"""
                  create table if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_table_name}(
                      mac_address string,
                      gym_id string,
                      login_time long,
                      logout_time long
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_table_name}")

    def create_bpm_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.bpm_table_name}
                  (
                   device_id long,
                   heartrate double,
                   time long
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.bronze_schema_name}.{self.bpm_table_name}")
    
    def create_user_profile_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.user_profile_table_name}
                  (
                    user_id string,
                    update_type string,
                    timestamp long,
                    dob string,
                    sex string,
                    gender string,
                    first_name string,
                    last_name string,
                    address STRUCT<street_address: STRING, city: STRING, state: STRING, zip: STRING>
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.bronze_schema_name}.{self.user_profile_table_name}")
    
    def create_workout_session_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.workout_session_table_name}
                  (
                      user_id string,
                      workout_id string,
                      timestamp long,
                      action string,
                      session_id int
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.bronze_schema_name}.{self.workout_session_table_name}")

    

    def create_bronze_layer_tables(self):
        self.create_bronze_layer_schema()
        self.create_device_registration_table()
        self.create_gym_attendance_table()
        self.create_bpm_table()
        self.create_user_profile_table()
        self.create_workout_session_table() 
        print("Bronze layer tables created successfully ! \n")


# Create the checkpoint volumes
class  CreateBronzeLayerCheckpointDirs(SetupBronzeLayer):
    def __init__(self):
        super().__init__()

    def create_device_registration_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/device_registration/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_table_name}_checkpoint")
    
    def create_gym_attendance_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/gym_attendance/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_table_name}_checkpoint")

    def create_bpm_table_checkpoint(self):    
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.bpm_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/bpm/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.bpm_table_name}_checkpoint")
    
    def create_user_profile_table_checkpoint(self):                 
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.user_profile_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/user_profile/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.user_profile_table_name}_checkpoint")
    
    def create_workout_session_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.workout_session_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/workout_session/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.workout_session_table_name}_checkpoint")


    def create_bronze_layer_checkpoint_volumes(self):
        self.create_device_registration_table_checkpoint()
        self.create_gym_attendance_table_checkpoint()
        self.create_bpm_table_checkpoint()
        self.create_user_profile_table_checkpoint()
        self.create_workout_session_table_checkpoint() 
        print("Bronze layer checkpoint volumes created successfully ! \n")

class CreateLandingZoneVolumes(SetupBronzeLayer):
    def __init__(self):
        super().__init__()

    def create_device_registration_landing_dir(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_landing_zone_name} location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-data/landing_zone/devices/' ")
                
        print(f"Landing Zone Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.device_registration_table_name}")
    
    def create_gym_attendance_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_landing_zone_name} location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-data/landing_zone/gym/' ")
                
        print(f"Landing Zone Volume Created - {self.catalog_name}.{self.bronze_schema_name}.{self.gym_attendance_table_name}")


    def create_landing_dir(self):
        self.create_device_registration_landing_dir()
        self.create_gym_attendance_table_checkpoint()
        print("Bronze layer Landing Zones created successfully ! ")


In [0]:
tables = CreateBronzeLayerTables()
tables.create_bronze_layer_tables()

checkpoints = CreateBronzeLayerCheckpointDirs()
checkpoints.create_bronze_layer_checkpoint_volumes()

landingZones = CreateLandingZoneVolumes()
landingZones.create_landing_dir()

Schema Created - dev.swatch_raw

Table Created - dev.swatch_raw.device_registration
Table Created - dev.swatch_raw.gym_attendance
Table Created - dev.swatch_raw.bpm
Table Created - dev.swatch_raw.user_profile
Table Created - dev.swatch_raw.workout_session
Bronze layer tables created successfully ! 

Checkpoint Volume Created - dev.swatch_raw.device_registration_checkpoint
Checkpoint Volume Created - dev.swatch_raw.gym_attendance_checkpoint
Checkpoint Volume Created - dev.swatch_raw.bpm_checkpoint
Checkpoint Volume Created - dev.swatch_raw.user_profile_checkpoint
Checkpoint Volume Created - dev.swatch_raw.workout_session_checkpoint
Bronze layer checkpoint volumes created successfully ! 

Landing Zone Volume Created - dev.swatch_raw.device_registration
Landing Zone Volume Created - dev.swatch_raw.gym_attendance
Bronze layer Landing Zones created successfully ! 
